# Example 10 - ROI pipeline (HC features)

In this notebook, we'll set up ROI pipeline:

(1) Apply Sv threshold, given a range, use this threshold to detect ROIs
(2) For each ROI, match with annotations, compute IoU, get labels for ROIs
(3) For each ROI, save as numpy array, in filename, add contextual information (depth, relative altitude, lat/lon)

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pickle

from src.read_echogram import EchogramReader
from src.detect_ROI import ROIDetector
from src.ROI_features import FeatureExtractor
from src.transform_annotations import AnnotationTransformer
from src.match_annotations import OverlapAnnotation

from multiprocessing import Pool

## Step 1. Basic setting

In this step, we'll set up filenames (raw, bot) dir, threshold range

In [2]:
# mode: train, val, test

In [3]:
mode = "train"

In [4]:
# load raw + bot paths
pkl_dir = "pkl/"
with open(pkl_dir + f"{mode}_li.pickle", "rb") as handle:
    data_li = pickle.load(handle)

In [5]:
# figure dir
fig_dir = "figures/"

In [6]:
# threshold range
threshold_li = [-54, -56, -58, -60, -62, -64, -66, -68, -70, -72, -74, -76, -78, -80] 
# freq selection
freq_li = [18, 38, 120, 200]
# kernel size
kernel_size = 3
# iou threshold
iou_threshold = 0.5

In [7]:
# annotations
with open(pkl_dir + 'annotations_dict_new_p4.pickle', 'rb') as handle:
    annotations_dict = pickle.load(handle)

## Step 2. ROI detection

In this step, we'll detect ROIs with each Sv threshold. 

In [8]:
# for each Sv threshold
    # for each filename
        # detect ROI
        # for each ROI, crop / save npy + get features
        # for each ROI, overlap with annotations + get labels (change to IoU, 0.5 threshold)
        # save in different folders

In [9]:
def roi_threshold(a):
    i, j = a # [raw_path, bot_path]
    # read echogram
    echogram = EchogramReader(i, j, freq_li)
    filename, Sv_npy, surface_idx, bottom_idx, time, depth, positions = echogram()
    print(filename)
    # select annotations
    flag = False
    if filename in annotations_dict:
        annotations_idx, labels = annotations_dict[filename]
        flag = True  
    # Sv thresheld
    all_contour_features = []
    for threshold in threshold_li:
        # (1) detect ROIs
        roi = ROIDetector(filename, Sv_npy, surface_idx, bottom_idx, fig_dir, threshold, kernel_size)
        img_shape, contours = roi()
        # (2) get contextual features
        features = FeatureExtractor(filename, contours, Sv_npy, bottom_idx, time, depth, positions)
        contours_sel, contours_features = features() 
        # contours_features: a list of dict
        # (3) overlap with annotations, get labels
        if flag == True:
            overlap = OverlapAnnotation(filename, img_shape, annotations_idx, labels, contours_sel, fig_dir)
            contours_labels = overlap.compute_iou(iou_threshold)
            for idx, contour in enumerate(contours_features):
                contour['label'] = contours_labels[idx] # add
                contour['threshold'] = threshold
        else:
            for idx, contour in enumerate(contours_features):
                contour['label'] = 0 # 0 - unlabeled, TODO: change to 5
                contour['threshold'] = threshold
        all_contour_features.extend(contours_features)
    return all_contour_features

In [10]:
pool = Pool(10)
res_li = pool.map(roi_threshold, data_li) # list of [raw_path, bot_path]

D20191001-T204744
D20191113-T083150
D20191003-T044219
D20191031-T045858
D20191109-T150801
D20191021-T204829
D20191109-T154858
D20190930-T235233
D20191014-T003421
D20190926-T091313
D20191109-T134607
D20191001-T233426
D20191111-T165715
D20191104-T124305
D20191110-T190705
D20191017-T145540
D20191003-T021135


Process ForkPoolWorker-7:
Traceback (most recent call last):
  File "/home/yazh0781/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/yazh0781/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yazh0781/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/yazh0781/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-9-a528eca8148f>", line 17, in roi_threshold
    img_shape, contours = roi()
  File "/home/yazh0781/Fisheries/pipeline/src/detect_ROI.py", line 25, in __call__
    Sv_npy = np.array([signal.medfilt2d(i, 3) for i in self.Sv_npy])
  File "/home/yazh0781/Fisheries/pipeline/src/detect_ROI.py", line 25, in <listcomp>
    Sv_npy = np.array([signal.medfilt2d(i, 3) for i in self.Sv_npy])
  File "/home/yazh0781/ana

KeyboardInterrupt: 

Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/home/yazh0781/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/yazh0781/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yazh0781/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/yazh0781/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-9-a528eca8148f>", line 20, in roi_threshold
    contours_sel, contours_features = features()
  File "/home/yazh0781/Fisheries/pipeline/src/ROI_features.py", line 27, in __call__
    Sv_npy = np.array([signal.medfilt2d(i, 3) for i in self.Sv_npy])
  File "/home/yazh0781/Fisheries/pipeline/src/ROI_features.py", line 27, in <listcomp>
    Sv_npy = np.array([signal.medfilt2d(i, 3) for i in self.Sv_npy])
  Fil

D20191013-T095943
D20191104-T193349
D20191107-T192619


In [ ]:
res = []
for sublist in res_li:
    if sublist == None:
        continue
    for item in sublist:
        res.append(item)
df_roi_features = pd.DataFrame(res)

In [ ]:
df_roi_features.to_pickle(pkl_dir + 'df_roi_features_thresholds.pkl')

In [ ]:
pool.close()
pool.join()

In [ ]:
# 'Unclassified regions': 1, 'krill_schools': 2, 'fish_school': 3, 'AH_School': 4
# 50 echograms - 1 day
# add break, early stopping! -> start: 9:15 pm
# annotations to cv2, fail, Umat for array, Use, try except instead!
# est. 6 hours